### Instruction 

- use github codesapce since it will setup all the tools for you, https://github.com/codespaces/new?hide_repo_select=true&ref=main&repo=599293758&machine=standardLinux32gb&devcontainer_path=.devcontainer%2Fdevcontainer.json&location=WestUs2

- az login --use-device-code
- az account set --subscription "e1b2a578-b466-409c-9466-cf5a7852d9ae"
- azd auth login
- azd up
- then give the parameter names , for openai select sweden centeral

### prepdocs.py
the **prepdocs.py** code : https://github.com/Azure-Samples/azure-search-openai-demo/blob/main/scripts/prepdocs.py is main for data processing . The script is designed to prepare documents by extracting content from PDFs, splitting the content into sections, uploading to Azure Blob Storage, and indexing in an Azure AI Search index. It uses various Azure services and custom libraries to accomplish these tasks. this code use all python libaray and classes in this folder https://github.com/Azure-Samples/azure-search-openai-demo/tree/main/scripts/prepdocslib . pdfparser.py decides whether to compute embeddings (text representations) using either Azure OpenAI services or  OpenAI services. If image searching is enabled, it retrieves the necessary Azure AI Vision key and sets up image embeddings. mor info : https://github.com/Azure-Samples/azure-search-openai-demo/blob/main/docs/data_ingestion.md

 #### 1-pdfparser.py
- **prepdocs.py**  use **pdfparser.py** from prepdocslib folder .**pdfparser.py**  determines the appropriate PDF parser to use based on whether local or Azure services are specified. basicly this code  provides two ways to parse a PDF and extract its content: one for local parsing with PyPDF and another using Azure document inteligent for more advanced analysis including table extraction.The table_to_html class method: Converts a DocumentTable object from Azure AI Document Intelligence into an HTML table string. This method properly formats headers and spans for rows and columns, and escapes HTML characters in cell content.

#### 2-textsplitter.py
- **prepdocs.py**  use **textsplitter.py** from prepdocslib folder .**textsplitter.py** is more complex and does the actual work of splitting the text. It has various parameters that define sentence endings, word breaks, the maximum length of a section, a limit for searching for sentence endings, and an overlap size for the sections. It also has a verbose mode for logging and a flag to indicate whether image embeddings are being used.Special care is taken to handle tables within the text; if a section ends with an unclosed table, the splitting process adjusts the start of the next section to include the table, avoiding splitting tables across sections.
In summary, this code is designed to split the text content of documents into manageable sections that can be processed by embedding models, while maintaining the integrity of sentences and tables

#### 3-strategy.py
- **prepdocs.py**  use **strategy.py** from prepdocslib folder .**strategy.py** have two classes: SearchInfo and Strategy.SearchInfo class:This class represents a connection to an Azure Search service. Strategy class:defines a strategy for ingesting documents into the Azure Search service

#### 4-filestrategy.py
- **prepdocs.py**  use **filestrategy.py** from prepdocslib folder .**filestrategy.py** is Strategy for ingesting documents into a search service from files stored either locally or in a data lake storage account. It initializes with multiple components:
   - list_file_strategy: For listing files to be processed.
   - blob_manager: For managing blobs (binary large objects) like uploading and removing.
   - pdf_parser: For parsing PDF documents.
   - text_splitter: For splitting text into sections or pages.
   - document_action: To specify the action to be taken (Add, Remove, or RemoveAll).
   - embeddings: For handling textual embeddings, optional and specific to OpenAI embeddings.
   - image_embeddings: For handling image embeddings, also optional.
   - search_analyzer_name, use_acls, and category: Additional optional parameters for configuring the search.
- **filestrategy.py** use https://github.com/Azure-Samples/azure-search-openai-demo/blob/main/scripts/prepdocslib/searchmanager.py too
   - **searchmanager.py** This code is a Python module that interacts with the Azure Search service to manage the indexing and searching of documents. <u>so all index configuration loactaed here.</u>
      -  The Section class represents a section of a page that will be stored in the search service, used for context by the Azure OpenAI service.
      - The SearchManager class is responsible for managing the search service. It can create search indexes and update or remove sections stored in these indexes. The class uses SearchInfo for settings and can optionally work with text and image embeddings provided by OpenAIEmbeddings.
      - The create_index async method creates a search index with defined fields. These fields include simple fields like id, category, sourcepage, and sourcefile, as well as complex fields for embeddings like embedding and imageEmbedding if images are included.
      - The search index is equipped with advanced configurations like semantic settings and vector search, using the HNSW (Hierarchical Navigable Small World) algorithm for efficient similarity search in high-dimensional spaces.https://learn.microsoft.com/en-us/azure/search/vector-search-ranking
      - The update_content async method is used to upload documents to the search index in batches. It handles the creation of document embeddings and the association of image embeddings if provided. more detail about embedding is in embeddings.py
      - The remove_content async method removes documents from the search index. If a specific path is provided, it removes only documents related to that path; otherwise, it removes all documents.
      - The code is designed to handle large-scale operations by breaking down the content into batches and using asynchronous operations for efficient processing.
      - The module is structured to work with Azure's search capabilities, leveraging AI-enhanced indexing and searching for documents, potentially including features like semantic understanding and image content search.

### App Backend
the primary backend code you'll want to customize is the app/backend/approaches folder, which contains the classes powering the Chat and Ask tabs. Each class uses a different RAG (Retrieval Augmented Generation) approach, which include system messages that should be changed to match your data

#### 1- Chat approach
The chat tab uses the approach programmed in **chatreadretrieveread.py**.chatreadretrieveread.py use classes and codes in approach folder like approach.py and chatapproach.py

 - It uses the OpenAI ChatCompletion API to turn the user question into a good search query.
 - It queries Azure AI Search for search results for that query (optionally using the vector embeddings for that query).
 - It then combines the search results and original user question, and asks OpenAI ChatCompletion API to answer the question based on the sources. It includes the last 4K of message history as well (or however many tokens are allowed by the deployed model).

**chatreadretrieveread.py**
- you can change system_message_chat_conversation 
- the default is top 3 document
- The query process in the search within the ChatReadRetrieveReadApproach class follows a multi-step approach to retrieve relevant information from a search index provided by Azure AI Search. Here's a more detailed explanation of how the query process works:
    - User Query Input: The process starts with a user's question or query as part of the chat history. This is the information that the user wants to know or get assistance with.
    - Generate Search Query: The chat history, which includes the user's last question, is processed to create a system prompt that will be used to instruct the OpenAI model.This system prompt is passed to the OpenAI chat completion API along with the model parameters to generate an optimized search query. This is done by the OpenAI model interpreting the user's question and converting it into a more structured and keyword-rich search query that can be used to retrieve relevant documents from the search index.
    - Search Document Index: With the optimized search query generated, the ChatReadRetrieveReadApproach class interacts with the Azure AI Search index using the SearchClient.Depending on the retrieval mode (text, vectors, or hybrid), the search can be conducted purely based on text, using vector embeddings for semantic search, or a combination of both. If vector embeddings are used, the class will compute a text embedding for the query which represents the semantic meaning of the query in a vector space. This helps in retrieving documents that are semantically related to the query. The search client then executes the search with the optimized query (and vectors if applicable) along with any specified filters. The filters can be based on authorization claims or other criteria to narrow down the search results.
    - Retrieve Results: The search results, which are documents from the Azure AI Search index that match the query, are retrieved. The class extracts the content from these documents and prepares it for the next step. This includes any semantic captions if the use_semantic_captions flag is set.
    - Generate Contextual Response: The retrieved documents' content, along with the original user query and conversation history, are used to generate a final response. This is again done through the OpenAI chat completion API, which takes into account the provided content and generates a response that is relevant and contextual.

By following this process, the ChatReadRetrieveReadApproach class effectively turns a user's natural language query into a structured search operation and uses the results of that search to inform a contextually relevant response back to the user.

Let's delve deeper into each step to understand the details of the code:
   - Initialization: The Approach class is instantiated with:
       - search_client: An Azure SearchClient to interact with Azure Cognitive Search services.
       - openai_client: An OpenAI client to use OpenAI's language model.
       - auth_helper: An instance of AuthenticationHelper to handle security filters based on user roles and permissions.
       - embedding_deployment and embedding_model: Configuration for OpenAI's embeddings to create vector representations of text.
       - Other configurations such as query language, query speller, and OpenAI host.

   - Building Filters: The build_filter method creates a string that Azure Search can use to filter out documents based on certain criteria like category or security constraints. It ensures that the search results are relevant and adhere to access permissions.
   - Search: The search method performs the document retrieval using the Azure SearchClient. Depending on whether semantic ranking is enabled (use_semantic_ranker) and if captions are to be used (use_semantic_captions), it will modify the search accordingly. It also handles pagination of results.
   - Source Content Compilation: The get_sources_content method constructs a formatted string representation of the retrieved documents, including proper citation formatting based on whether it's a text or image source.
   - Chat Approach: The ChatApproach class handles chat interactions and is abstract, meaning it requires specific implementations to define certain properties and methods.
   - Running the Chat: the run method checks if streaming is enabled (stream parameter) and either processes the chat in real-time or in a single batch, returning the final response or an asynchronous generator for streaming.
   - Chat Read-Retrieve-Read Approach:The ChatReadRetrieveReadApproach class specifically implements the chat approach with the following steps:
       - Read: Processes the user's question and formulates a search query.
       - Retrieve: Fetches documents relevant to the search query.
       - Read: Uses the retrieved documents to generate a comprehensive answer to the user's query.
  - Generating Search Query:In run_until_final_call, a search query is formulated using OpenAI's chat completion, which takes into account the entire conversation history.
  - Retrieving Documents: Once the query is generated, documents are retrieved from Azure Search. If vector-based retrieval is specified, embeddings for the query are computed to enhance the search.
  - Generating Response: The chat completion from OpenAI is then used to generate a final response. The system prompt is dynamically constructed based on whether follow-up questions are suggested and whether an override prompt is provided.
  - Serialization: The serialize_for_results method in the Document class is used to serialize document data into a dictionary, trimming down long embeddings for readability.
  - Additional Details:
       - ThoughtStep: A dataclass that represents a step in the thought process, which can be used for logging or debugging.
       - ChatReadRetrieveReadApproach: This class defines the run_until_final_call method with both overloads for streaming and non-streaming modes, orchestrating the multi-step process.
       - Handling Responses: The code handles the generation of follow-up questions, extraction of search queries from chat completions, and the management of conversation history.
       - Message Building: The get_messages_from_history method in ChatApproach constructs a list of messages for OpenAI's chat completion, including previous chat history and new user input.

Throughout the code, various configurations and parameters are used to refine the search and response generation process to ensure that the chatbot provides accurate and relevant answers while adhering to permissions and access controls. The intricate interplay between Azure Search and OpenAI's language models creates a sophisticated system capable of handling complex user queries.


#### 2 Chat with vision

If you followed the instructions in docs/gpt4v.md https://github.com/Azure-Samples/azure-search-openai-demo/blob/main/docs/gpt4v.md to enable the GPT-4 Vision model and then select "Use GPT-4 Turbo with Vision", then the chat tab will use the chatreadretrievereadvision.py approach instead. This approach is similar to the chatreadretrieveread.py approach, with a few differences:

 - Step 1 is the same as before, except it uses the GPT-4 Vision model instead of the default GPT-3.5 model.
 - For this step, it also calculates a vector embedding for the user question using the Computer Vision vectorize text API, and passes that to the Azure AI Search to compare against the imageEmbeddings fields in the indexed documents. For each matching document, it downloads the image blob and converts it to a base 64 encoding.
  - When it combines the search results and user question, it includes the base 64 encoded images, and sends along both the text and images to the GPT4 Vision model (similar to this documentation example https://platform.openai.com/docs/guides/vision). The model generates a response that includes citations to the images, and the UI renders the base64 encoded images when a citation is clicked.

#### 3 Ask tab

The ask tab uses the approach programmed in **retrievethenread.py**.

- It queries Azure AI Search for search results for the user question (optionally using the vector embeddings for that question).
- It then combines the search results and user question, and asks OpenAI ChatCompletion API to answer the question based on the sources.

The system_chat_template variable is currently tailored to the sample data since it starts with "You are an intelligent assistant helping Contoso Inc employees with their healthcare plan questions and employee handbook questions." Change that to match your data.